In [ ]:
from IPython.display import HTML

HTML('''<script>
 function code_toggle() {
   if (code_shown){
     $('div.input').hide('500');
     $('#toggleButton').val('Show Code')
   } else {
     $('div.input').show('500');
     $('#toggleButton').val('Hide Code')
   }
   code_shown = !code_shown
 }

 $( document ).ready(function(){
   code_shown=false;
   $('div.input').hide()
 });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>'''
   )


In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh', 'matplotlib')

In [ ]:
def plotspikes(s, times, offset=2):
    '''Plot a spike raster with colors.
    
    Parameters
    ----------
    s : array
        Array of shape (N, nTimes) containing 0s and 1s
    times : array
        Array of shape (nTimes) containing the times of each frame.
    offset : float
        How much to offset each neuron's spike train by
        
    Returns
    -------
    Holoviews Overlay
        An Holoview overlay of spike scatters
    '''
    N, nT = s.shape
    
    # get spike frames and ids
    spike_ids, spike_frames = np.where(s)
    
    # generate figure
    out = hv.Overlay()
    for i in range(N):
        # get spike times
        sp_times = times[spike_frames[spike_ids==i]]
        
        # add to figure
        out *= hv.Scatter(zip(sp_times, np.ones(len(sp_times))*i*offset), kdims='Time (s)')
        
    return out

In [ ]:
#### Parameters
N = 100 # number of neurons
dt = 1e-3
Tend = 10
times = np.arange(0, Tend, dt)
nT = len(times)
nInputs = 10 # number of different input strengths
inputs = np.linspace(0.5, 1.5, nInputs) # input strengths
sigma = .1 # noise level (std)

In [ ]:
#### Simulate with Euler
V = np.zeros((nInputs, N, nT))
V_nospiking = np.zeros_like(V)
s = np.zeros_like(V)
for j in range(nInputs): # loop over different inputs
    for i in range(1, nT): # loop over time
        # generate noise        
        eta = np.random.randn(N)
        
        # update voltages without spiking mechanism
        V[j, :, i] = V[j, :, i-1] + dt*(-V[j, :, i-1]+ inputs[j])+ sigma*np.sqrt(dt)*eta
        
        # update voltages with spiking mechanism
        V_nospiking[j, :, i] = V_nospiking[j, :, i-1] + dt*(-V_nospiking[j, :, i-1]+ inputs[j])+ sigma*np.sqrt(dt)*eta

        # check which neurons are spiking (threshold of 1)
        to_spike = V[j, :, i] > 1
        if len(to_spike) > 0:
            V[j, to_spike, i] = 0 # reset to 0
            s[j, to_spike, i] = 1 # update spikes

### Plot voltages with distributions, no spiking

In [ ]:
%%output backend='bokeh'
# pick an example neuron to plot voltages for
n = 3

# bins for voltage distribution plotting
nbins = 100
bins = np.linspace(V_nospiking.min(), V_nospiking.max()+0.2, nbins)

# prepare figure
figA = hv.Overlay()
figB = hv.Overlay()
vdim = 'Voltage (a.u.)'
tdim = 'Time (s)'
fig_thresh_H = hv.HLine(1).opts(color='k', line_dash='dashed', line_width=1)
fig_thresh_V = hv.VLine(1).opts(color='k', line_dash='dashed', line_width=1)

# generate figure
for j in range(nInputs): # loop over different input strengths
    fig = hv.Overlay()
    
    # calculate and plot histogram
    histy, histx = np.histogram(V_nospiking[j, :, 4000:], bins=bins)
    fig_hist = hv.Curve(zip(histy/histy.sum(), histx), vdims=vdim)
    
    # plot example voltage trace
    fig = hv.Curve(zip(times, V_nospiking[j, n, :]), vdims=vdim, kdims=tdim)
    
    # combine figures
    figA *= fig
    figB *= fig_hist.opts(width=100, xaxis=None)
    
figA*fig_thresh_H+figB*fig_thresh_H

Note that the voltage distributions on the right are across many trials, not just for the example trace on the left..

### Plot voltages with distributions, with spiking

In [ ]:
%%output backend='bokeh'
# pick an example neuron to plot voltages for
n = 3

# bins for voltage distribution plotting
nbins = 100
bins = np.linspace(V.min(), V.max()+0.2, nbins)

# prepare figure
figA = hv.Overlay()
figB = hv.Overlay()
vdim = 'Voltage (a.u.)'
tdim = 'Time (s)'
fig_thresh_H = hv.HLine(1).opts(color='k', line_dash='dashed', line_width=1)
fig_thresh_V = hv.VLine(1).opts(color='k', line_dash='dashed', line_width=1)

# generate figure
for j in range(nInputs): # loop over different input strengths
    fig = hv.Overlay()
    
    # calculate and plot histogram
    histy, histx = np.histogram(V[j, :, 4000:], bins=bins)
    fig_hist = hv.Curve(zip(histy/histy.sum(), histx), vdims=vdim)
    
    # plot example voltage trace
    fig = hv.Curve(zip(times, V[j, n, :]), vdims=vdim, kdims=tdim)
    
    # combine figures
    figA *= fig
    figB *= fig_hist.opts(width=100, xaxis=None)
    
figA*fig_thresh_H+figB*fig_thresh_H

### Interactive version without thresholds

In [ ]:
%%output backend='bokeh'
# choose example neuron for the voltage trace
n = 3

# bins for voltage distribution plotting
nbins = 100
bins = np.linspace(V_nospiking.min(), V_nospiking.max()+0.2, nbins)

# prepare figure
frames = {}
vdim = 'Voltage (a.u.)'
tdim = 'Time (s)'
fig_thresh_H = hv.HLine(1).opts(color='k', line_dash='dashed', line_width=1)
fig_thresh_V = hv.VLine(1).opts(color='k', line_dash='dashed', line_width=1)

# generate figure
for j in range(nInputs):
    fig = hv.Overlay()
    
    # calculate histogram and plot
    histy, histx = np.histogram(V_nospiking[j, :, 2000:], bins=bins)
    fig_hist = hv.Curve(zip(histy/histy.sum(), histx), vdims=vdim)
    
    # plot example voltage
    fig = hv.Curve(zip(times, V_nospiking[j, n, :]), vdims=vdim, kdims=tdim)
    
    # add figure to frames
    frames[inputs[j]] = fig*fig_thresh_H + fig_hist.opts(width=100, xaxis=None)*fig_thresh_H

# combine all in interactive plot    
hv.HoloMap(frames, kdims=['Input strength']).collate()

### Interactive version with thresholds

In [ ]:
%%output backend='bokeh'
# choose example neuron for the voltage trace
n = 3

# bins for voltage distribution plotting
nbins = 100
bins = np.linspace(V.min(), V.max()+0.2, nbins)

# prepare figure
frames = {}
vdim = 'Voltage (a.u.)'
tdim = 'Time (s)'
fig_thresh_H = hv.HLine(1).opts(color='k', line_dash='dashed', line_width=1)
fig_thresh_V = hv.VLine(1).opts(color='k', line_dash='dashed', line_width=1)

# generate figure
for j in range(nInputs):
    fig = hv.Overlay()
    
    # calculate histogram and plot
    histy, histx = np.histogram(V[j, :, 2000:], bins=bins)
    fig_hist = hv.Curve(zip(histy/histy.sum(), histx), vdims=vdim)
    
    # plot example voltage
    fig = hv.Curve(zip(times, V[j, n, :]), vdims=vdim, kdims=tdim)
    
    # add figure to frames
    frames[inputs[j]] = fig*fig_thresh_H + fig_hist.opts(width=100, xaxis=None)*fig_thresh_H

# combine all in interactive plot    
hv.HoloMap(frames, kdims=['Input strength']).collate()

### Example traces and spikes

In [ ]:
%%opts Curve {-axiswise} Scatter {-axiswise}  [yaxis=None]
%%opts Overlay [yaxis=None] {+axiswise}
Nsamples = range(20) # which neurons to actually plot
fig_A = hv.HoloMap({inputs[i]: hv.Overlay([hv.Curve(zip(times, V[i, n, :]+n*2), kdims='Time (s)') for n in Nsamples]) for i in range(nInputs)}, kdims=['Input strength'])
fig_B = hv.HoloMap({inputs[i]: plotspikes(s[i, Nsamples, :], times) for i in range(nInputs)}, kdims=['Input strength'])
fig_A+fig_B

Left: voltage traces. Right: spike raster plot.